#### [aio](https://pymotw.com/3/asyncio/)

[tornado](https://www.rddoc.com/doc/Tornado/4.4.2/zh/gen/)

In [ ]:
asyncio — Asynchronous I/O, event loop, and concurrency tools

Asynchronous Concurrency Concepts


大多数使用其他并发模型的程序都是线性编写的，并且依赖于语言运行时或操作系统的底层线程或进程管理来根据需要更改上下文.基于asyncio的应用程序需要应用程序代码来显式处理上下文更改，使用正确的技术取决于理解几个相关的概念.


asyncio提供的框架集中在一个事件循环，一个负责有效处理I/O事件，系统事件和应用程序上下文变化的第一个类对象。提供了几个循环实现，以有效利用操作系统功能.虽然通常会自动选择合理的默认值，但也可以从应用程序中选择特定的事件循环实现.这在Windows下很有用，例如，某些循环类以一种可能会影响网络I / O效率的方式添加对外部进程的支持.


应用程序与事件循环显式交互以注册要运行的代码，并在资源可用时让事件循环对应用程序代码进行必要的调用.例如，网络服务器打开套接字，然后注册它们以在输入事件发生时被告知.事件循环会在有新的传入连接或有数据要读取时通知服务器代码. 当没有工作可以被完成在当前上下文中时，应用程序代码期望在短时间内再次获取控制权。例如，如果没有更多的数据要从套接字中读取，服务器应该将控制权返回给事件循环。

将控制权返回给事件循环的机制取决于Python的协同程序，这种特殊的功能放弃了对调用者的控制而不会丢失其状态。协程类似于生成器函数，事实上生成器可以用于在早于3.5的Python版本中实现协程，而不需要原生支持协程对象.asyncio还为协议和传输提供了一个基于类的抽象层，用于使用回调来编写代码，而不是直接编写协程.在基于类和协程的模型中，通过重新进入事件循环显式地改变上下文取代了Python线程实现中的隐式上下文变化。

future是表示尚未完成的工作结果的数据结构. 事件循环可以监视Future对象被设置完成，允许应用程序的一部分等待另一部分完成一些工作.除了future外，asyncio还包括其他并发原语，如锁和信号量。

Task是Future的一个子类，它知道如何Wrap和管理协程的执行.可以使用事件循环来安排任务，以便在所需的资源可用时运行，并生成其他协同程序可以使用的结果。


##### 1 Cooperative Multitasking with Coroutines(协程多任务协同工作)

状态保持和恢复.

协程是一种为并发操作而设计的语言结构.协程函数在调用时创建协程对象，然后调用者可以使用协程的send()方法运行该函数的代码.协程可以使用另一个协程的await关键字来暂停执行.当它暂停时，协程状态保持不变，允许它在下一次被唤醒时恢复到原来的状态。

1.Starting a Coroutine


2.Returning Values from Coroutines

In [ ]:
import asyncio


async def coroutine():
    print('in coroutine')
    return 'result'


event_loop = asyncio.get_event_loop()
try:
    return_value = event_loop.run_until_complete(
        coroutine()
    )
    print('it returned: {!r}'.format(return_value))
finally:
    event_loop.close()

3.Chaining Coroutines

一个协程可以启动另一个协程并等待结果。这使得将任务分解成可重用的部分变得更容易。以下示例有两个阶段必须按顺序执行，但可以与其他操作同时运行。

In [ ]:
import asyncio


async def outer():
    print('in outer')
    print('waiting for result1')
    result1 = await phase1()
    print('waiting for result2')
    result2 = await phase2(result1)
    return (result1, result2)


async def phase1():
    print('in phase1')
    return 'result1'


async def phase2(arg):
    print('in phase2')
    return 'result2 derived from {}'.format(arg)


event_loop = asyncio.get_event_loop()
try:
    return_value = event_loop.run_until_complete(outer())
    print('return value: {!r}'.format(return_value))
finally:
    event_loop.close()

In [1]:
# $ python3 asyncio_coroutine_chain.py

# in outer
# waiting for result1
# in phase1
# waiting for result2
# in phase2
# return value: ('result1', 'result2 derived from result1')

4 Generators Instead of Coroutines

协程功能是asyncio设计的关键组成部分。它们提供了一种语言结构，用于停止执行程序的一部分，保留该调用的状态，并在以后重新进入状态，这些都是并发框架的重要功能。


Python 3.5引入了新的语言特性来定义这种使用async def本地化的协程，并使用await来产生控制，而asyncio的例子利用了这个新特性。早期版本的Python 3可以使用由asyncio.coroutine（）装饰器包装的生成器函数，并从yield from 中获得相同的效果

###### 2 Scheduling Calls to Regular Functions 调度对常规函数的调用

除了管理协程和I / O回调之外，asyncio事件循环还可以根据循环中保存的定时器值来调度常规函数

###### 2.1 Scheduling a Callback “Soon” 

如果回调的时间不重要，可以使用call_soon（）来调度循环的下一次迭代. 任何扩展的的位置参数在被调用时传递给回调函数. 要将关键字参数传递给回调，请使用functools模块中的partial（）

#####  2.2 Scheduling a Callback with a Delay

##### 2.3 Scheduling a Callback for a Specific Time

##### 3 Producing Results Asynchronously （异步生成结果）

##### 4 Executing Tasks Concurrently (并发执行任务)

##### 5 Composing Coroutines with Control Structures

##### 6 Synchronization Primitives

##### 7 Asynchronous I/O with Protocol Class Abstractions

##### 8 Asynchronous I/O Using Coroutines and Streams

Using SSL
Interacting with Domain Name Services
Working with Subprocesses
Receiving Unix Signals
Combining Coroutines with Threads and Processes
Debugging with asyncio